# Training CNN Models

We trained the optimized **VGG16** and **VGG16 with CBAM attention mechanism** models with the [7,000 unique sky
photos](https://doi.org/10.5281/zenodo.7830131) of the occurrence or non-occurrence of falling meteors.

In [ ]:
import pandas as pd
import numpy as np
import math
import os
import glob
import shutil

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from utils.dataset import DataHandler
from utils.models import VGGModel
from utils import evaluation
from pathlib import Path
import argparse

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

if not os.path.exists('models'):
    os.mkdir('models')
    
parser = argparse.ArgumentParser()
args = parser.parse_args("")

In [ ]:
# Load the 7.000 meteor dataset
args.path_prefix = 'data/cropped24_images/'
args.classes = ['nonmeteor', 'meteor'] # zero -> nonmeteor; one -> meteor
args.image_size = (224, 224)
args.seed = 2023

np.random.seed(args.seed)
tf.random.set_seed(args.seed)

data_handler = DataHandler(args, path_prefix=args.path_prefix)
df_images = data_handler.load_dataset(shuffle=True)

In [ ]:
# Auxiliary functions
def visualize_metrics(fold_no, acc, f1, sp, sn, tss, far, er, loss):
    print(f'Test score for fold {fold_no+1}:')
    print(f'\tAcc: {acc}%')
    print(f'\tF1_: {f1}%')
    print(f'\tSp_: {sp}%')
    print(f'\tSn_: {sn}%')
    print(f'\tTSS: {tss}%')
    print(f'\tFAR: {far}%')
    print(f'\tER_: {er}')
    print(f'\tLoss: {loss}')
    
    
def predict(model, data_gen):
    y_test = data_gen.classes
    y_pred = np.argmax(model.predict(data_gen), axis=1)
    
    acc = evaluation.accuracy(y_test, y_pred)
    f1 = evaluation.f1_score(y_test, y_pred)
    sp = evaluation.specificity(y_test, y_pred)
    sn = evaluation.sensitivity(y_test, y_pred)
    tss = evaluation.true_skill_statistic(y_test, y_pred)
    far = evaluation.false_alarm_ratio(y_test, y_pred)
    er = evaluation.error_rate(y_test, y_pred)
    
    return acc, f1, sp, sn, tss, far, er

In [ ]:
%%time

# Training
inputs = df_images['id']
targets = df_images['label']

datagen = ImageDataGenerator(
    rescale=1./255,
)

# Configurations
image_size = (224, 224, 3)
model_name = 'cbam' # baseline or cbam
n_dense_layers = 16
activation_func = 'relu'
dropout = 0.2

batchsize_training = 8
batchsize_finetune = 32

adamw_lr_training = 1e-05
adamw_lr_finetune = 1e-04
adamw_wd_training = 0.0005
adamw_wd_finetune = 0.0005

# Define per-fold score containers
train_acc_per_fold = []
train_f1_per_fold  = []
train_sp_per_fold  = []
train_sn_per_fold  = []
train_tss_per_fold = []
train_far_per_fold = []
train_er_per_fold  = []
train_loss_per_fold = []

valid_acc_per_fold = []
valid_f1_per_fold  = []
valid_sp_per_fold  = []
valid_sn_per_fold  = []
valid_tss_per_fold = []
valid_far_per_fold = []
valid_er_per_fold  = []
valid_loss_per_fold = []

test_acc_per_fold = []
test_f1_per_fold  = []
test_sp_per_fold  = []
test_sn_per_fold  = []
test_tss_per_fold = []
test_far_per_fold = []
test_er_per_fold  = []
test_loss_per_fold = []

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=5, shuffle=False)
for fold_no, (train, test) in enumerate(kfold.split(inputs, targets)):
  
    # == Generate a print ==
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no+1} ...')

    # == Generate data ==
    df_train = df_images.iloc[train]
    df_test = df_images.iloc[test]
    df_train, df_valid = train_test_split(df_train, test_size=0.2, stratify=df_train['label'], random_state=args.seed)
    train_gen, valid_gen, test_gen = data_handler.generate_data(datagen, df_train, df_valid, df_test, batch_size=batchsize_training)

    # == Checkpointer ==
    checkpointer = tf.keras.callbacks.ModelCheckpoint(f'models/vgg_model_fold{fold_no+1}.h5', 
                                                      monitor="val_loss",
                                                      save_best_only=True, 
                                                      save_weights_only=False)
    
    # == Create model ==
    vgg_builder = VGGModel(image_size=image_size, vgg_model='16')
    if model_name == 'baseline':
        model = vgg_builder.create_bachnorm_model(place_between_activation=False,
                                                  n_layers=n_dense_layers,
                                                  activation=activation_func,
                                                  dropout=dropout)
    elif model_name == 'cbam':
        model = vgg_builder.create_cbam_model(n_layers=n_dense_layers,
                                              activation=activation_func,
                                              dropout=dropout)
    
    # == Training ==
    optimizer = tf.keras.optimizers.experimental.AdamW(weight_decay=adamw_wd_training, learning_rate=adamw_lr_training)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_gen,
                        validation_data=valid_gen,
                        epochs=10)
    
    # == Fine tune ==
    print('Fine tuning ...')
    model.trainable = True
    train_gen, valid_gen, test_gen = data_handler.generate_data(datagen, df_train, df_valid, df_test, batch_size=batchsize_finetune)
    optimizer = tf.keras.optimizers.experimental.AdamW(weight_decay=adamw_wd_finetune, learning_rate=adamw_lr_finetune)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_gen,
                        validation_data=valid_gen,
                        epochs=50,
                        callbacks=[checkpointer])

    # == Generate metrics ==
    print('Model evaluation ...')
    model.load_weights(f'models-cropped/vgg_model_fold{fold_no+1}.h5')
    train_gen, valid_gen, test_gen = data_handler.generate_data(datagen, df_train, df_valid, df_test, shuffle=False)
    
    # Train set
    loss, acc_eval = model.evaluate(train_gen, verbose=0)
    acc, f1, sp, sn, tss, far, er = predict(model, train_gen)
    assert round(acc_eval, 2) == round(acc, 2)
    train_acc_per_fold.append(acc)
    train_f1_per_fold.append(f1)
    train_sp_per_fold.append(sp)
    train_sn_per_fold.append(sn)
    train_tss_per_fold.append(tss)
    train_far_per_fold.append(far)
    train_er_per_fold.append(er)
    train_loss_per_fold.append(loss)
    
    # Val set
    loss, acc_eval = model.evaluate(valid_gen, verbose=0)
    acc, f1, sp, sn, tss, far, er = predict(model, valid_gen)
    assert round(acc_eval, 2) == round(acc, 2)
    valid_acc_per_fold.append(acc)
    valid_f1_per_fold.append(f1)
    valid_sp_per_fold.append(sp)
    valid_sn_per_fold.append(sn)
    valid_tss_per_fold.append(tss)
    valid_far_per_fold.append(far)
    valid_er_per_fold.append(er)
    valid_loss_per_fold.append(loss)
    
    # Test set
    loss, acc_eval = model.evaluate(test_gen, verbose=0)
    acc, f1, sp, sn, tss, far, er = predict(model, test_gen)
    assert round(acc_eval, 2) == round(acc, 2)
    test_acc_per_fold.append(acc)
    test_f1_per_fold.append(f1)
    test_sp_per_fold.append(sp)
    test_sn_per_fold.append(sn)
    test_tss_per_fold.append(tss)
    test_far_per_fold.append(far)
    test_er_per_fold.append(er)
    test_loss_per_fold.append(loss)
    
    # Visualize test generalization metrics
    visualize_metrics(fold_no, acc, f1, sp, sn, tss, far, er, loss)


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(test_acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {test_loss_per_fold[i]} - Accuracy: {test_acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(test_acc_per_fold)} (+- {np.std(test_acc_per_fold)})')
print(f'> Loss: {np.mean(test_loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
# Compile results
results_df = pd.DataFrame({'TRAIN_ACC':train_acc_per_fold, 'VALID_ACC':valid_acc_per_fold, 'TEST_ACC':test_acc_per_fold, 
                           'TRAIN_F1':train_f1_per_fold, 'VALID_F1':valid_f1_per_fold, 'TEST_F1':test_f1_per_fold,
                           'TRAIN_SP':train_sp_per_fold, 'VALID_SP':valid_sp_per_fold, 'TEST_SP':test_sp_per_fold,
                           'TRAIN_SN':train_sn_per_fold, 'VALID_SN':valid_sn_per_fold, 'TEST_SN':test_sn_per_fold,
                           'TRAIN_TSS':train_tss_per_fold, 'VALID_TSS':valid_tss_per_fold, 'TEST_TSS':test_tss_per_fold,
                           'TRAIN_FAR':train_far_per_fold, 'VALID_FAR':valid_far_per_fold, 'TEST_FAR':test_far_per_fold,
                           'TRAIN_ER':train_er_per_fold, 'VALID_ER':valid_er_per_fold, 'TEST_ER':test_er_per_fold,
                           'TRAIN_LOSS':train_loss_per_fold, 'VALID_LOSS':valid_loss_per_fold, 'TEST_LOSS':test_loss_per_fold})
results_df

In [ ]:
# Save to disk
results_df.to_csv(f'results/training_{model_name}_results.csv', index=False)